<a href="https://www.kaggle.com/code/narberalgamma/case-study-s-p-500-leveraged-etfs?scriptVersionId=140804985" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

---
## Title: Case Study:S&P 500 Leveraged ETFs: A Case Study on Risk and Return
## Author: Edgar M.
## Date: 6/11/23
---
## Hypothetical Scenario
As a junior financial analyst working for an asset management firm.<br>
Asset manager asked me to lead a project for a brand-new client — this<br>
will involve everything from defining the business task all the way<br> 
through presenting data-driven recommendations. I will ask the right<br>
questions, identify all appropriate datasets and ensure their integrity,<br>
conduct analysis, create compelling data visualizations, and give<br>
recommendations to the client based on my findings. 

## The Case Study Roadmap 

- Ask
- Prepare
- Process 
- Analyze 
- Share 
- Act

## Ask
Ask phase involves defining the business problem and question that the<br>
analysis will address. Identifying the key stakeholders who will be<br>
involved in the analysis. Identifying the metrics that will be used to<br>
measure the success of the project. It also involves identifying the data<br>
sources that will be used in the analysis and determining the quality and<br>
completeness of the data.<br> 

### Key Tasks
- Identify the business task. What is the problem that is being solved?

The client is interested in investing in S&P 500 leveraged ETF's. The<br>
business task is to help the client understand the risk and return<br> 
profile of the S&P 500 leveraged ETFs. The regular unleverage S&P 500 ETF<br>
and leveraged S&P 500 ETF's have different risk and return profiles.<br> 
Hence the problem being solved is the assessment of the risk and<br>
return profile of the unleveraged S&P 500 vs the S&P 500 Leveraged ETFs.

- Determine key stakeholders.

The client is a high net worth individual who is interested in investing<br>
in S&P 500 Leveraged ETFs. Project will also be presented to the asset<br>
manager and the client's financial advisor.<br>

- What metrics will be used to measure data to achieve objective?

One metric that will be used to measure the objective is DCA (Dollar<br>
Cost Averaging). DCA reduces risk through the use of periodic purchases<br>
of a target asset. Rather than one large purchase, which can expose the<br>
investor to the risk of timing the market, DCA spreads out the risk over<br>
time. Monthly DCA will be used to measure financial performance. Metric<br>
that will be used to measure financial risk is Beta (β). Beta is a<br>
measure of the volatility of a security or portfolio compared to the<br>
market as a whole.<br>

- How can your insights help your client make decisions?

Based on the total return, Beta, and DCA analysis, the client will be<br>
able to make an informed financial decision.<br>

- What type of data will be appropriate for analysis?

There will be three datasets used in the analysis. The first dataset will<br>
be the S&P 500 index data. GSPC tracks the performance of the S&P 500<br>
index. The second dataset will be a 2X S&P 500 leveraged ETF. SSO tracks<br>
the performance of the S&P 500 index with a leverage factor of 2. The<br>
third dataset will be a 3X S&P 500 leveraged ETF. UPRO tracks the<br>
performance of the S&P 500 index with a leverage factor of 3.

- Where will data be obtained?

The data will be obtained quantmod package in R. The quantmod package<br>
provides a framework for quantitative financial modeling and trading. 


## Prepare
Prepare involves gathering data used in the analysis.

In [ ]:
#load nessary packages
library(quantmod)
library(readr)
library(dplyr)
library(lubridate)
library(ggplot2) 
library(scales)
library(PerformanceAnalytics)

Download GSPC S&P 500 data.<br>
Start date of S&P 500.<br>
Set end date as current system date<br>
Download data and store as an xts object named GSPC<br>
Convert xts (time-series object) to data frame

In [ ]:
start_date_SP500 <- as.Date("1957-03-04")
end_date_SP500 <- Sys.Date()
getSymbols("^GSPC", src = "yahoo", from = start_date_SP500, to = end_date_SP500)
SP500_data_unclean <- as.data.frame(GSPC)

Download and save SSO ETF.<br>
Start date of SSO.<br>
Set end date as current system date.<br>
Download data and store as an xts object named SSO<br>
Convert xts (time-series object) to data frame.

In [ ]:
start_date_SSO <- as.Date("2006-03-04")
end_date_SSO <- end_date <- Sys.Date()
getSymbols("SSO", src = "yahoo", from = start_date_SSO, to = end_date_SSO)
SSO_data_unclean <- as.data.frame(SSO)

Download and save UPRO ETF.<br>
Very fisrt date of UPRO.<br>
Set end date as current system date.<br>
Download data and store as an xts object named UPRO<br>
Convert xts (time-series object) to data frame.

In [ ]:
start_date_UPRO <- as.Date("2009-06-24")
end_date_UPRO <- end_date <- Sys.Date()
getSymbols("UPRO", src = "yahoo", from = start_date_UPRO, to = end_date_UPRO)
UPRO_data_unclean <- as.data.frame(UPRO)

## Process
Process involves cleaning and transforming data for analysis.

### Clean GSPC (S&P 500) Data
Assign row names to the "Date" column.<br>
Remove the row names.

In [ ]:
SP500_data_unclean$Date <- rownames(SP500_data_unclean)
rownames(SP500_data_unclean) <- NULL

Create a new column "Trading day". seq_len(nrow(SP500_data_unclean))<br>
creates a sequence of integers from 1 to the number of rows in the data<br>
frame. Finally, the resulting sequence is assigned to the Trading_day<br>
column.

In [ ]:
SP500_data_unclean$Trading_day <- seq_len(nrow(SP500_data_unclean))

Select Trading_day, Date, GSPC.Close, rename columns and mutate Date to date format.

In [ ]:
SP500_data_clean <- select(SP500_data_unclean, Trading_day, Date, GSPC.Close) %>% 
  rename(SP500_Closing_Price = GSPC.Close) %>%
  mutate(Date = parse_date_time(Date, orders = c("mdy", "dmy", "ymd")))

View the fisrt 6 rows of the resulting SP500_data_clean dataframe.

In [ ]:
head(SP500_data_clean)

### Clean SSO Data (2X S&P 500 Leveraged ETF)
Assign row names to the "Date" column.<br>
Remove the row names.

In [ ]:
SSO_data_unclean$Date <- rownames(SSO_data_unclean)
rownames(SSO_data_unclean) <- NULL

Create a new column "Trading day". seq_len(nrow(SSO_data_unclean)) creates<br>
a sequence of integers from 1 to the number of rows in the data frame.<br>
Finally, the resulting sequence is assigned to the Trading_day column.

In [ ]:
SSO_data_unclean$Trading_day <- seq_len(nrow(SSO_data_unclean))

Select Trading_day, Date, SSO.Close, rename columns and mutate Date to date format.

In [ ]:
SSO_data_clean <- select(SSO_data_unclean, Trading_day, Date, SSO.Close) %>% 
  rename(SSO_Closing_Price = SSO.Close) %>%
  mutate(Date = parse_date_time(Date, orders = c("mdy", "dmy", "ymd")))

View the fisrt 6 rows of the resulting SSO_data_clean dataframe.

In [ ]:
head(SSO_data_clean)

### Clean UPRO Data (3X S&P 500 Leveraged ETF)
Assign row names to the "Date" column.<br>
Remove the row names.

In [ ]:
UPRO_data_unclean$Date <- rownames(UPRO_data_unclean)
rownames(UPRO_data_unclean) <- NULL

Create a new column "Trading day."seq_len(nrow(UPRO_data_unclean)) creates<br>
a sequence of integers from 1 to the number of rows in the data frame.<br>
Finally, the resulting sequence is assigned to the Trading_day column.

In [ ]:
UPRO_data_unclean$Trading_day <- seq_len(nrow(UPRO_data_unclean))

Select Trading_day, Date, UPRO.Close, rename columns and mutate Date to date format.

In [ ]:
UPRO_data_clean <- select(UPRO_data_unclean, Trading_day, Date, UPRO.Close) %>% 
  rename(UPRO_Closing_Price = UPRO.Close) %>%
  mutate(Date = parse_date_time(Date, orders = c("mdy", "dmy", "ymd")))

View the fisrt 6 rows of the resulting UPRO_data_clean dataframe.

In [ ]:
head(UPRO_data_clean)

## Analysis and Share
Analysis involves using exploratory data analysis and visualization to<br>
explore relationships in the data. Share is the act of creating<br>
visualizations and sharing them with others.<br>

Set GSPC as a hypothetical S&P 500 ETF investment. Leveraged GSPC ETFs<br>
will be compared to GSPC.<br>  

Set monthly investment amount ($).

In [ ]:
Monthly_Investment_USD <- 10

### Number of Shares of S&P 500 ETF Purchased on a Monthly Basis
Calculate the investment amount based on 1957-03-05 closing price.

In [ ]:
first_day_price_SP500 <- SP500_data_clean$SP500_Closing_Price[2]
Monthly_Share_1_SP500 <- Monthly_Investment_USD / first_day_price_SP500

Create data frame for first month's investment.

In [ ]:
first_day_date_SP500 <- SP500_data_clean$Date[2]
first_day_price_SP500 <- SP500_data_clean$SP500_Closing_Price[2]
Monthly_Share_1_df_SP500 <- data.frame(Date = first_day_date_SP500, 
  Shares = Monthly_Share_1_SP500, 
  SP500_Closing_Price = first_day_price_SP500)

Subset data from 1957-04-01 onwards.

In [ ]:
SP500_data_filtered <- SP500_data_clean %>%
  filter(Date >= as.Date("1957-04-01", format = "%Y-%m-%d"))

Create a vector of the first trading day for each month.

In [ ]:
first_of_month_SP500 <- SP500_data_filtered %>% 
  group_by(year(Date), month(Date)) %>% 
  slice(1) %>% 
  pull(Date)

Calculate the investment amount for each month.

In [ ]:
investment_amount_SP500 <- Monthly_Investment_USD / SP500_data_filtered$SP500_Closing_Price[match(first_of_month_SP500, SP500_data_filtered$Date)]
closing_prices_SP500 <- SP500_data_filtered$SP500_Closing_Price[match(first_of_month_SP500, SP500_data_filtered$Date)]

Create the data frame.

In [ ]:
Monthly_Shares_2_df_SP500 <- data.frame(Date = first_of_month_SP500, 
  Shares = investment_amount_SP500, SP500_Closing_Price = closing_prices_SP500)

Combine Monthly_Share_1_df and Monthly_Shares_2_df into Monthly_Shares_SP500

In [ ]:
Monthly_Shares_SP500 <- rbind(Monthly_Share_1_df_SP500, Monthly_Shares_2_df_SP500)

View the first 6 rows of the resulting Monthly_Shares_SP500 dataframe.

In [ ]:
head(Monthly_Shares_SP500)

Create a new data frame to store cumulative shares and corresponding<br>
closing prices.

In [ ]:
Sum_Shares_SP500 <- data.frame(
  # The first column is a sequence of dates, starting from the initial date
  Date = rep(c(as.Date("1957-03-05"), first_of_month_SP500), each = 1),
  # The second column is a sequence of zeroes to store cumulative shares
  Shares = numeric(length(Monthly_Shares_SP500$Shares)),
  # The third column is a sequence of zeroes to store corresponding closing prices
  SP500_Closing_Price = numeric(length(Monthly_Shares_SP500$Shares))
)

# Loop through each index of the Monthly_Shares$Shares vector
for (i in seq_along(Monthly_Shares_SP500$Shares)) {
  # Calculate the cumulative sum of shares up to the current index and store it in the Shares column
  Sum_Shares_SP500[i, "Shares"] <- sum(Monthly_Shares_SP500[1:i, "Shares"])
  # Assign the corresponding SP500_Closing_Price value to the current index
  Sum_Shares_SP500[i, "SP500_Closing_Price"] <- Monthly_Shares_SP500[i, "SP500_Closing_Price"]
}

Insert a new column called "Total_USD" into Sum_Shares dataframe.<br>
Multiply (Shares) by (SP500_Closing_Price).

In [ ]:
Sum_Shares_SP500$Total_USD <- Sum_Shares_SP500$Shares * Sum_Shares_SP500$SP500_Closing_Price

View the fisrt 6 rows of the resulting Sum_Shares dataframe.

In [ ]:
head(Sum_Shares_SP500)

### Daily Percent Change
#### Daily Percent Change for SSO (2x Leveraged S&P 500 ETF)
First trading day is 6/21/2006 for SSO.

Create the new data frame with Date column starting from the second row.

In [ ]:
#note Year-month-day
SSO_Daily_Percent_Change_Datedf1 <- data.frame(Date = SSO_data_clean$Date[-1])

Calculate daily percent change.<br>
Exclude the first element of closing prices.

In [ ]:
closing_pricesSSO <- SSO_data_clean$SSO_Closing_Price[-1]  

Exclude the last element of closing prices.

In [ ]:
previous_pricesSSO <- SSO_data_clean$SSO_Closing_Price[-length(SSO_data_clean$SSO_Closing_Price)]

Calculate the daily percent change.

In [ ]:
daily_changeSSO <- (closing_pricesSSO * 100 / previous_pricesSSO) - 100  

Create the new data frame.

In [ ]:
SSO_Daily_Percent_Changedf2 <- data.frame(Daily_Percent_Change = daily_changeSSO)

Combine the two data frames.

In [ ]:
SSO_Daily_Percent_Changedf <- cbind(SSO_Daily_Percent_Change_Datedf1, SSO_Daily_Percent_Changedf2)

View the first 6 rows of the resulting SSO_Daily_Percent_Changedf dataframe.

In [ ]:
head(SSO_Daily_Percent_Changedf)

#### Daily Percent Change for UPRO (3x Leveraged S&P 500 ETF)
First trading day is 6/21/2009 for UPRO.<br>

Create the new data frame with Date column starting from the second row.

In [ ]:
#note Year-month-day
UPRO_Daily_Percent_Change_Datedf1 <- data.frame(Date = UPRO_data_clean$Date[-1])

Calculate daily percent change.<br>
Exclude the first element of closing prices.

In [ ]:
closing_pricesUPRO <- UPRO_data_clean$UPRO_Closing_Price[-1]

Exclude the last element of closing prices.

In [ ]:
previous_pricesUPRO <- UPRO_data_clean$UPRO_Closing_Price[-length(UPRO_data_clean$UPRO_Closing_Price)]

Calculate the daily percent change.

In [ ]:
daily_changeUPRO <- (closing_pricesUPRO * 100 / previous_pricesUPRO) - 100

Create the new data frame.

In [ ]:
UPRO_Daily_Percent_Changedf2 <- data.frame(Daily_Percent_Change = daily_changeUPRO)

Combine the two data frames.

In [ ]:
UPRO_Daily_Percent_Changedf <- cbind(UPRO_Daily_Percent_Change_Datedf1, UPRO_Daily_Percent_Changedf2)

View the first 6 rows of the resulting UPRO_Daily_Percent_Changedf dataframe.

In [ ]:
head(UPRO_Daily_Percent_Changedf)

### S&P 500 Daily Percent Change Dataframes Corresponding to The Dates SSO and UPRO.
#### S&P 500 Daily Percent Change Dataframe Corresponding to The Dates of SSO.
First date from SSO_Daily_Percent_Change_Date_df1 minus one.

In [ ]:
first_dateSSO <- as.Date(head(SSO_Daily_Percent_Change_Datedf1$Date, n = 1), format = "%Y-%m-%d") - 1

Filter for dates greater than first_date_SSO.

In [ ]:
SP500_data_cleanSSO <- SP500_data_clean %>%
    filter(Date >= first_dateSSO)

Fisrt element is the closing price that corresponds the date of the data<br>
frame "SSO_Daily_Percent_Change_Date_df1"

In [ ]:
closing_pricesSP500 <- SP500_data_cleanSSO$SP500_Closing_Price[-1] 

Exclude the last element of closing prices.

In [ ]:
previous_pricesSP500 <- SP500_data_cleanSSO$SP500_Closing_Price[-length(SP500_data_cleanSSO$SP500_Closing_Price)]

Calculate the daily percent change.

In [ ]:
daily_changeSP500 <- (closing_pricesSP500 * 100 / previous_pricesSP500) - 100  

Create the new data frame.

In [ ]:
SP500_Daily_Percent_Changedf2 <- data.frame(Daily_Percent_Change = daily_changeSP500)  

Combine the two data frames

In [ ]:
SP500_Daily_Percent_Change_SSOdf <- cbind(SSO_Daily_Percent_Change_Datedf1, SP500_Daily_Percent_Changedf2)

View the first 6 rows of the resulting SP500_Daily_Percent_Change_SSOdf<br>
dataframe. S&P 500 Daily Percent Change Dataframe corresponding to the<br>
dates of SSO. 

In [ ]:
head(SP500_Daily_Percent_Change_SSOdf)

#### S&P 500 Daily Percent Change Dataframe Corresponding to The Dates of UPRO.
First date from UPRO_Daily_Percent_Change_Date_df1 

In [ ]:
first_dateUPRO <- as.Date(head(UPRO_Daily_Percent_Change_Datedf1$Date, n = 1), format = "%Y-%m-%d") 

Convert the "Date" column to Date format.<br>
This line of code does not create a new variable but rather updates the<br>
existing Date column within the SP500_Daily_Percent_Change_SSO_df data frame.

In [ ]:
SP500_Daily_Percent_Change_SSOdf$Date <- as.Date(SP500_Daily_Percent_Change_SSOdf$Date, format = "%Y-%m-%d")

Filter for dates greater than or equal to first_date_UPRO

In [ ]:
SP500_Daily_Percent_Change_UPROdf <- SP500_Daily_Percent_Change_SSOdf %>%
  filter(Date >= first_dateUPRO)

View the first 6 rows of the resulting SP500_Daily_Percent_Change_UPROdf<br>
dataframe. S&P 500 Daily Percent Change Dataframe corresponding to the<br>
dates of UPRO.

In [ ]:
head(SP500_Daily_Percent_Change_UPROdf)

### Calculate Daily Leverage and Median Daily Leverage for SSO and UPRO.
#### Calculate Daily Leverage for SSO.
Create the SSO_Lev data frame. Need to replace infinite values for SSO_LevDiv.

In [ ]:
SSO_LevDiv <- data.frame(SSO_Leverage = SSO_Daily_Percent_Changedf2 / SP500_Daily_Percent_Changedf2)   

Replace Infinities in data with 2.

In [ ]:
SSO_LevDiv <- do.call(data.frame,lapply(SSO_LevDiv, function(x) replace(x, is.infinite(x), 2)))

Create the SSO_Lev data frame with the desired column name.

In [ ]:
SSOLev <- data.frame(SSO_Leverage = SSO_LevDiv$Daily_Percent_Change)

View the first 6 rows of the resulting SSOLev dataframe.<br>
Daily Leverage for SSO.

In [ ]:
head(SSOLev)

#### Calculate Daily Leverage for UPRO.
Makes new data frame called SP500_Daily_Percent_Change_df3.<br>
Filters out the Date column from SP500_Daily_Percent_Change_UPRO_df.

In [ ]:
SP500_Daily_Percent_Changedf3 <- SP500_Daily_Percent_Change_UPROdf %>% 
  select('Daily_Percent_Change')

Create the UPRO_Lev data frame. Need to replace infinite values for UPRO_Leverage.

In [ ]:
UPRO_LevDiv <- data.frame(UPRO_Leverage = UPRO_Daily_Percent_Changedf2 / SP500_Daily_Percent_Changedf3)

Replace Infinities in data with 3.

In [ ]:
UPRO_LevDiv <- do.call(data.frame,lapply(UPRO_LevDiv, function(x) replace(x, is.infinite(x), 3)))

Create the UPRO_Lev data frame with the desired column name.

In [ ]:
UPROLev <- data.frame(UPRO_Leverage = UPRO_LevDiv$Daily_Percent_Change)

View the first 6 rows of the resulting UPROLev dataframe.<br>
Daily Leverage for UPRO.

In [ ]:
head(UPROLev)

#### Filter Out Median Daily Leverage For SSO and UPRO.
Median Daily Leverage for SSO.

In [ ]:
MedianSSO <- SSOLev %>%
  summarise(Median_Lev_SSO = median(SSO_Leverage))

head(MedianSSO)

Median Daily Leverage for UPRO.

In [ ]:
MedianUPRO <- UPROLev %>%
  summarise(Median_Lev_UPRO = median(UPRO_Leverage))

head(MedianUPRO)

### Create Daily Percent Change For S&P 500 (1x, 2x, 3x).
#### Create Daily Percent Change Dataframe For 1X S&P 500.
Create the new data frame with date column starting from the second row.

In [ ]:
#note Year-month-day
SP500_Daily_Percent_Change_Datedf1 <- data.frame(Date = SP500_data_clean$Date[-1])

Calculate daily percent change.<br>
Exclude the first element of closing prices.

In [ ]:
closing_pricesSP500 <- SP500_data_clean$SP500_Closing_Price[-1] 

Exclude the last element of closing prices.

In [ ]:
previous_pricesSP500 <- SP500_data_clean$SP500_Closing_Price[-length(SP500_data_clean$SP500_Closing_Price)]  

Calculate the daily percent change.

In [ ]:
daily_changeSP500 <- (closing_pricesSP500 * 100 / previous_pricesSP500) - 100

Create the new data frame.

In [ ]:
SP500_Daily_Percent_Changedf2 <- data.frame(Daily_Percent_Change = daily_changeSP500)

Combine the two data frames.

In [ ]:
SP500_Daily_Percent_Changedf <- cbind(SP500_Daily_Percent_Change_Datedf1, SP500_Daily_Percent_Changedf2)

#### Create Daily Percent Change Dataframe For S&P 500 (1x, 2x, 3x).
To calculate the daily percent change for the S&P 500 (1x, 2x, 3x)<br>
dataframes, one might be tempted to multiply the daily percent change of<br>
the 1X S&P 500 by 2 and 3, respectively. However, this method is not<br>
entirely accurate due to the daily rebalancing of leverage ETFs. Which<br>
introduces a tracking error between the non-leveraged and leveraged ETFs.<br>
This error causes a small deviation from the target leverage of 2 and 3.<br>
Therefore, a more accurate approach is to calculate the median leverage<br>
for the 2X and 3X datasets, then multiply it by the daily percent change<br>
of the 1X S&P 500. This method takes into account the varying leverage<br>
ratios of the ETFs and provides a more accurate representation of the<br>
daily percent change for the S&P 500 (2x, 3x).

In [ ]:
Lev_Percent_df <- SP500_Daily_Percent_Changedf %>%
  # Calculate the product of Median_Lev_SSO and Daily_Percent_Change
  mutate(SP500_2X_DailyPercent = MedianSSO$Median_Lev_SSO * Daily_Percent_Change,
  # Calculate the product of Median_Lev_UPRO and Daily_Percent_Change
         SP500_3X_DailyPercent = MedianUPRO$Median_Lev_UPRO * Daily_Percent_Change) %>%
  # Select specific columns to keep in the resulting data frame
  select(Date, Daily_Percent_Change, SP500_2X_DailyPercent, SP500_3X_DailyPercent) %>% 
  # Rename selected columns
  rename(SP500_DailyPercent = Daily_Percent_Change)

View the first 6 rows of the resulting Lev_Percent_df dataframe.

In [ ]:
head(Lev_Percent_df)

### Create Closing Price Dataframe For S&P 500 (1x, 2x, 3x).
#### Create Closing Price Dataframe For 1X S&P 500.
Filter out fisrt row from SP500_data_clean.

In [ ]:
SP500_data_clean2 <- SP500_data_clean[-1,]

#### Create Closing Price Dataframe for 2X Leveraged S&P 500.
Calculates the daily closing prices of a leveraged S&P 500 ETF by<br>
applying a cumulative product function to the daily percentage changes in<br>
the ETF's value, starting from the first day's price.

In [ ]:
SP500_2XClosingPrice_df2 <- data.frame(SP500_2XClosingPrice = Reduce(function(x, y) x * (100 + y) / 100, Lev_Percent_df$SP500_2X_DailyPercent, accumulate = TRUE, init = first_day_price_SP500))

Remove fisrt row from SP500_2XClosingPrice_df2.

In [ ]:
SP500_2XClosingPrice_df2 <- SP500_2XClosingPrice_df2[-1,]

Make dataframe and rename column as SP500_2XClosing_Price.

In [ ]:
SP500_2XClosingPrice_df <- data.frame(SP500_2XClosingPrice_df2)  %>% 
  rename(SP500_2XClosing_Price = SP500_2XClosingPrice_df2)

#### Create Closing Price Dataframe For 3X Leveraged S&P 500.
Same procedure as SP500_2XClosingPrice_df2, but now for 3X leverage.

In [ ]:
SP500_3XClosingPrice_df2 <- data.frame(SP500_3XClosingPrice = Reduce(function(x, y) x * (100 + y) / 100, Lev_Percent_df$SP500_3X_DailyPercent, accumulate = TRUE, init = first_day_price_SP500))

Remove fisrt row from SP500_3XClosingPrice_df2.

In [ ]:
SP500_3XClosingPrice_df2 <- SP500_3XClosingPrice_df2[-1,]

Make dataframe and rename column to SP500_3XClosing_Price.

In [ ]:
SP500_3XClosingPrice_df <- data.frame(SP500_3XClosingPrice_df2)  %>% 
  rename(SP500_3XClosing_Price = SP500_3XClosingPrice_df2)

#### Dataframe Closing Price for S&P 500 (1x, 2x, 3x).
Combine SP500_data_clean2, SP500_2XClosingPrice_df, SP500_3XClosingPrice_df

In [ ]:
SP500_ALL_ClosingPrices <- cbind(SP500_data_clean2, SP500_2XClosingPrice_df, SP500_3XClosingPrice_df)

View the first 6 rows of the resulting SP500_ALL_ClosingPrices dataframe.

In [ ]:
head(SP500_ALL_ClosingPrices)

### Plot Closing Prices for S&P 500 (1x, 2x, 3x).
The line for 1X S&P 500 is barely noticible in the graph.2X S&P 500 and<br>
3X S&P 500 rise very quickly when compared to 1X S&P 500.3X S&P 500 rises<br>
the quickest of the three. Therefore it is best to graph in log scale.

In [ ]:
SP500_ALL_ClosingPrices_DateConvert <- SP500_ALL_ClosingPrices
SP500_ALL_ClosingPrices_DateConvert$Date <- as.Date(SP500_ALL_ClosingPrices_DateConvert$Date, format = "%Y-%m-%d")

SP500_ALL_ClosingPrices_plot <- ggplot(data = SP500_ALL_ClosingPrices_DateConvert, aes(x = Date, y = SP500_Closing_Price), group=1) +
  geom_line(aes(color = "SP500")) +
  geom_line(data = SP500_ALL_ClosingPrices_DateConvert, aes(x = Date, y = SP500_2XClosing_Price, color = "SP500_2X"), group = 1) +
  geom_line(data = SP500_ALL_ClosingPrices_DateConvert, aes(x = Date, y = SP500_3XClosing_Price, color = "SP500_3X"), group = 1) +
  scale_y_continuous(breaks = c(0,4000 ,50000,75000, 10000, 100000, 150000, 200000, 250000, 300000, 320000),
                     labels = c("0", "4K", "50K","75K", "10K", "100K", "150K", "200K", "250K", "300K", "320K")) +
  labs(y = element_text("Price USD", size = 16, face = "bold"), x = element_text("Date", size = 16, face = "bold")) +
  scale_x_date(date_breaks = "5 years", date_labels = "%Y") +
  scale_color_manual(values = c("SP500" = "red", "SP500_2X" = "blue", "SP500_3X" = "darkgreen"),
                     labels = c("SP500", "SP500_2X", "SP500_3X")) +
  guides(color = guide_legend(title = "Color")) +
  theme(plot.title = element_text(face = "bold", hjust = 0.5),
        panel.grid.minor.y = element_blank(),
        panel.grid.major.y = element_line(color = "gray", linetype = "dashed"),
        axis.text.x = element_text(size = 14),
        axis.text.y = element_text(size = 14)) +
  ggtitle("Price USD for SP500, SP500_2X, AND SP500_3X")


In [ ]:
options(repr.plot.width = 25, repr.plot.height = 20)

print(SP500_ALL_ClosingPrices_plot)

### Plot Closing Prices for S&P 500 (1x, 2x, 3x) in Log Scale.
Now in log scale, the graph is more readible. 

In [ ]:
SP500_ALL_ClosingPrices_plot_log10 <- ggplot(data = SP500_ALL_ClosingPrices_DateConvert, aes(x = Date, y = log10(SP500_Closing_Price)), group=1) +
  geom_line(aes(color = "SP500")) +
  geom_line(data = SP500_ALL_ClosingPrices_DateConvert, aes(x = Date, y = log10(SP500_2XClosing_Price), color = "SP500_2X"), group = 1) +
  geom_line(data = SP500_ALL_ClosingPrices_DateConvert, aes(x = Date, y = log10(SP500_3XClosing_Price), color = "SP500_3X"), group = 1) +
  scale_y_continuous(breaks = c(1.477, 2, 3, 3.602, 4, 4.875, 5, 5.301, 5.477),
                     labels = c("30" ,"100", "1K", "4K", "10K", "75K", "100K", "200K", "300K")) +
  labs(y = element_text("log10(Price USD)", size = 16, face = "bold"), x = element_text("Date", size = 16, face = "bold")) +
  scale_x_date(date_breaks = "5 years", date_labels = "%Y") +
  scale_color_manual(values = c("SP500" = "red", "SP500_2X" = "blue", "SP500_3X" = "darkgreen"),
                     labels = c("SP500", "SP500_2X", "SP500_3X")) +
  guides(color = guide_legend(title = "Color")) +
  theme(plot.title = element_text(face = "bold", hjust = 0.5),
        panel.grid.minor.y = element_blank(),
        panel.grid.major.y = element_line(color = "gray", linetype = "dashed"),
        axis.text.x = element_text(size = 14),
        axis.text.y = element_text(size = 14)) +
  ggtitle("log10(Price USD) for SP500, SP500_2X, AND SP500_3X")

In [ ]:
options(repr.plot.width = 25, repr.plot.height = 20)

print(SP500_ALL_ClosingPrices_plot_log10)

### Number of Shares of S&P 500 Leveraged ETF purchased on a Monthly Basis
Now that the closing prices for S&P 500 (1x, 2x, 3x) are known. The<br>
number of shares bought on a monthly basis for each leverage ETF can be<br>
calculated. The number of shares bought is based on Monthly_Investment_USD<br>
amount and the closing price for each leverage ETF. 

#### Number of Shares of S&P 500 2X Leveraged ETF purchased on a Monthly Basis
Calculate the investment amount based on 1957-03-05 price.

In [ ]:
first_day_price_SP5002X <- SP500_ALL_ClosingPrices$SP500_2XClosing_Price[1]
Monthly_Share_1_SP5002X <- Monthly_Investment_USD / first_day_price_SP5002X

Create data frame for first month's investment.

In [ ]:
first_day_date_SP5002X <- SP500_ALL_ClosingPrices$Date[1]
first_day_price_SP5002X <- SP500_ALL_ClosingPrices$SP500_2XClosing_Price[1]
Monthly_Share_1_df_SP5002X <- data.frame(Date = first_day_date_SP5002X, 
  Shares2X = Monthly_Share_1_SP5002X, 
  SP500_2XClosing_Price = first_day_price_SP5002X)

Subset data from 1957-04-01 onwards.

In [ ]:
SP500_data_filtered2X <- SP500_ALL_ClosingPrices %>%
  filter(Date >= as.Date("1957-04-01", format = "%Y-%m-%d"))

Create a vector of the first trading day for each month.

In [ ]:
first_of_month_SP5002X <- SP500_data_filtered2X %>% 
  group_by(year(Date), month(Date)) %>% 
  slice(1) %>% 
  pull(Date)

Calculate the investment amount for each month.

In [ ]:
investment_amount_SP5002X <- Monthly_Investment_USD / SP500_data_filtered2X$SP500_2XClosing_Price[match(first_of_month_SP5002X, SP500_data_filtered2X$Date)]
closing_prices_SP5002X <- SP500_data_filtered2X$SP500_2XClosing_Price[match(first_of_month_SP5002X, SP500_data_filtered2X$Date)]

Create data frame for monthly investment.

In [ ]:
Monthly_Shares_2_df_SP5002X <- data.frame(Date = first_of_month_SP5002X, 
  Shares2X = investment_amount_SP5002X, SP500_2XClosing_Price = closing_prices_SP5002X)

Combine Monthly_Share_1_df and Monthly_Shares_2_df into Monthly_Shares_SP5002X_df.

In [ ]:
Monthly_Shares_SP5002X <- rbind(Monthly_Share_1_df_SP5002X, Monthly_Shares_2_df_SP5002X)

Create a new data frame to store cumulative shares and corresponding closing prices.

In [ ]:
Sum_Shares_SP5002X <- data.frame(
  # The first column is a sequence of dates, starting from the initial date
  Date = rep(c(as.Date("1957-03-05"), first_of_month_SP5002X), each = 1),
  # The second column is a sequence of zeroes to store cumulative shares
  Shares2X = numeric(length(Monthly_Shares_SP5002X$Shares2X)),
  # The third column is a sequence of zeroes to store corresponding closing prices
  SP500_2XClosing_Price = numeric(length(Monthly_Shares_SP5002X$Shares2X))
)

# Loop through each index of the Monthly_Shares$Shares vector
for (i in seq_along(Monthly_Shares_SP5002X$Shares2X)) {
  # Calculate the cumulative sum of shares up to the current index and store it in the Shares2X column
  Sum_Shares_SP5002X[i, "Shares2X"] <- sum(Monthly_Shares_SP5002X[1:i, "Shares2X"])
  # Assign the corresponding SP500_Closing_Price value to the current index
  Sum_Shares_SP5002X[i, "SP500_2XClosing_Price"] <- Monthly_Shares_SP5002X[i, "SP500_2XClosing_Price"]
}

Insert a new column called "Total_USD" into Sum_Shares dataframe.
Multiply (Shares) by (SP500_Closing_Price).

In [ ]:
Sum_Shares_SP5002X$Total_USD2X <- Sum_Shares_SP5002X$Shares2X * Sum_Shares_SP5002X$SP500_2XClosing_Price

#### Number of shares of S&P 500 3X Leveraged ETF Purchased on a Monthly Basis.
Calculate the investment amount based on 1957-03-05 price.

In [ ]:
first_day_price_SP5003X <- SP500_ALL_ClosingPrices$SP500_3XClosing_Price[1]
Monthly_Share_1_SP5003X <- Monthly_Investment_USD / first_day_price_SP5003X

Create data frame for first month's investment

In [ ]:
first_day_date_SP5003X <- SP500_ALL_ClosingPrices$Date[1]
first_day_price_SP5003X <- SP500_ALL_ClosingPrices$SP500_3XClosing_Price[1]
Monthly_Share_1_df_SP5003X <- data.frame(Date = first_day_date_SP5003X, 
  Shares3X = Monthly_Share_1_SP5003X, 
  SP500_3XClosing_Price = first_day_price_SP5003X)

Subset data from 1957-04-01 onwards.

In [ ]:
SP500_data_filtered3X <- SP500_ALL_ClosingPrices %>%
  filter(Date >= as.Date("1957-04-01", format = "%Y-%m-%d"))

Create a vector of the first trading day for each month.

In [ ]:
first_of_month_SP5003X <- SP500_data_filtered3X %>% 
  group_by(year(Date), month(Date)) %>% 
  slice(1) %>% 
  pull(Date)

Calculate the investment amount for each month.

In [ ]:
investment_amount_SP5003X <- Monthly_Investment_USD / SP500_data_filtered3X$SP500_3XClosing_Price[match(first_of_month_SP5003X, SP500_data_filtered3X$Date)]
closing_prices_SP5003X <- SP500_data_filtered3X$SP500_3XClosing_Price[match(first_of_month_SP5003X, SP500_data_filtered3X$Date)]

Create data frame for monthly investment.

In [ ]:
Monthly_Shares_2_df_SP5003X <- data.frame(Date = first_of_month_SP5003X, 
  Shares3X = investment_amount_SP5003X, SP500_3XClosing_Price = closing_prices_SP5003X)

Combine Monthly_Share_1_df and Monthly_Shares_2_df into Monthly_Shares_SP5003X_df.

In [ ]:
Monthly_Shares_SP5003X <- rbind(Monthly_Share_1_df_SP5003X, Monthly_Shares_2_df_SP5003X)

Create a new data frame to store cumulative shares and corresponding closing prices.

In [ ]:
Sum_Shares_SP5003X <- data.frame(
  # The first column is a sequence of dates, starting from the initial date
  Date = rep(c(as.Date("1957-03-05"), first_of_month_SP5003X), each = 1),
  # The second column is a sequence of zeroes to store cumulative shares
  Shares3X = numeric(length(Monthly_Shares_SP5003X$Shares3X)),
  # The third column is a sequence of zeroes to store corresponding closing prices
  SP500_3XClosing_Price = numeric(length(Monthly_Shares_SP5003X$Shares3X))
)

# Loop through each index of the Monthly_Shares$Shares vector
for (i in seq_along(Monthly_Shares_SP5003X$Shares3X)) {
  # Calculate the cumulative sum of shares up to the current index and store it in the Shares2X column
  Sum_Shares_SP5003X[i, "Shares3X"] <- sum(Monthly_Shares_SP5003X[1:i, "Shares3X"])
  # Assign the corresponding SP500_Closing_Price value to the current index
  Sum_Shares_SP5003X[i, "SP500_3XClosing_Price"] <- Monthly_Shares_SP5003X[i, "SP500_3XClosing_Price"]
}

Insert a new column called "Total_USD" into Sum_Shares dataframe.<br>
Multiply (Shares) by (SP500_Closing_Price).

In [ ]:
Sum_Shares_SP5003X$Total_USD3X <- Sum_Shares_SP5003X$Shares3X * Sum_Shares_SP5003X$SP500_3XClosing_Price

#### Summary of Total USD Value for S&P 500 (1X, 2X, 3X).

In [ ]:
SP500_select <- Sum_Shares_SP500 %>%
  select(Date, Total_USD) %>%
  rename(Total_USD_SP500 = Total_USD) 
  
SP500_select2X <- Sum_Shares_SP5002X %>%
  select(Total_USD2X) %>%
  rename(Total_USD_SP5002X = Total_USD2X)  

SP500_select3X <- Sum_Shares_SP5003X %>%
  select(Total_USD3X) %>%
  rename(Total_USD_SP5003X = Total_USD3X) 

SP500_ALL <- cbind(SP500_select, SP500_select2X, SP500_select3X)

View first 6 rows of SP500_ALL. Dataframe shows the total USD value of<br>
the investment in S&P 500, 1X, 2X and 3X ETFs.

In [ ]:
head(SP500_ALL)

### Plot of Total USD Value for S&P 500, 1X, 2X, and 3X ETFs.
Graph shows the total USD value of the investment in S&P 500, 1X, 2X and<br>
3X ETFs. The investment amount is 10 dollars per month. The monthly investment<br>
amount doesn't matter. If the monthly amount is 100 dollars, then the total USD<br>
value will be 10 times higher. The graph retains the same shape and trend<br>
regardless of the monthly investment amount. 

In [ ]:
SP500_ALL_plot <- ggplot(data = SP500_ALL, aes(x = Date, y = Total_USD_SP500), group=1) +
  geom_line(aes(color = "SP500")) +
  geom_line(data = SP500_ALL, aes(x = Date, y = Total_USD_SP5002X, color = "SP500_2X"), group = 1) +
  geom_line(data = SP500_ALL, aes(x = Date, y = Total_USD_SP5003X, color = "SP500_3X"), group = 1) +
  scale_y_continuous(breaks = c(10, 100000, 1000000, 2000000, 3000000, 4000000, 5000000, 6000000, 7000000),
                     labels = c("10", "100K", "1M", "2M", "3M", "4M", "5M", "6M", "7M")) +
  labs(y = "Total USD", x = "Date") +
  scale_x_date(date_breaks = "5 years", date_labels = "%Y") +
  scale_color_manual(values = c("SP500" = "red", "SP500_2X" = "blue", "SP500_3X" = "darkgreen"),
                     labels = c("SP500", "SP500_2X", "SP500_3X")) +
  guides(color = guide_legend(title = "Color")) +
  theme(plot.title = element_text(face = "bold", hjust = 0.5),
        panel.grid.minor.y = element_blank(),
        panel.grid.major.y = element_line(color = "gray", linetype = "dashed"),
        axis.text.x = element_text(size = 14),
        axis.text.y = element_text(size = 14),
        axis.title.x = element_text(size = 16),
        axis.title.y = element_text(size = 16)) +
  ggtitle("Total USD for SP500, SP500_2X, AND SP500_3X")

In [ ]:
options(repr.plot.width = 25, repr.plot.height = 20)

print(SP500_ALL_plot)

As with "Price USD for SP500, SP500_2X, AND SP500_3X" graph, the total<br>
USD value rises exponentially. Therefore, it is best to graph in log<br>
scale. The graph is more readible in log scale.

In [ ]:
SP500_ALL_plot_log10 <- ggplot(data = SP500_ALL, aes(x = Date, y = Total_USD_SP500), group=1) +
  geom_line(aes(color = "SP500")) +
  geom_line(data = SP500_ALL, aes(x = Date, y = Total_USD_SP5002X, color = "SP500_2X"), group = 1) +
  geom_line(data = SP500_ALL, aes(x = Date, y = Total_USD_SP5003X, color = "SP500_3X"), group = 1) +
  scale_y_log10(breaks = c(10, 100, 1000, 10000, 100000, 200000, 1000000, 2000000, 3000000, 4000000, 5000000, 6000000, 7000000),
                labels = c("10", "100", "1K", "10K", "100K", "200K", "1M", "2M", "3M", "4M", "5M", "6M", "7M")) +
  labs(y = "Total USD Log Base 10", x = "Date") +
  scale_x_date(date_breaks = "5 years", date_labels = "%Y") +
  scale_color_manual(values = c("SP500" = "red", "SP500_2X" = "blue", "SP500_3X" = "darkgreen"),
                     labels = c("SP500", "SP500_2X", "SP500_3X")) +
  guides(color = guide_legend(title = "Color")) +
  theme(plot.title = element_text(face = "bold", hjust = 0.5),
        panel.grid.minor.y = element_blank(),
        panel.grid.major.y = element_line(color = "gray", linetype = "dashed"),
        axis.text.x = element_text(size = 14),
        axis.text.y = element_text(size = 14),
        axis.title.x = element_text(size = 16),
        axis.title.y = element_text(size = 16)) +
  ggtitle("log10(Total USD) for SP500, SP500_2X, AND SP500_3X")

In [ ]:
options(repr.plot.width = 25, repr.plot.height = 20)

print(SP500_ALL_plot_log10)

### Latest Total USD Value For S&P 500 (1X, 2X, 3X).
#### Latest Total USD Value For 1X S&P 500.

In [ ]:
#big.mark argument specifies the character to use as the separator for thousands
#digits argument specifies the number of decimal places to display
comma_format <- function(x) {
  comma(x, big.mark = ",", decimal.mark = ".", digits = 0)
}

In [ ]:
#return the date and Total_USD_SP500 value from the last row
Total_USD_SP500_latest <- SP500_ALL[nrow(SP500_ALL), c("Date", "Total_USD_SP500")]
# Format the Total_USD_SP500_latest value with commas and periods
Total_USD_SP500_latest_formatted <- comma_format(Total_USD_SP500_latest$Total_USD_SP500)
# Format the date in the Total_USD_SP500_latest data frame as a string in the format "YYYY-MM-DD"
Total_USD_SP500_latest_date <- format(Total_USD_SP500_latest$Date, "%Y-%m-%d")
# Print the formatted date and Total_USD_SP500_latest value to the console with the desired format
cat("Date Total_USD_SP500\n", Total_USD_SP500_latest_date, " ", Total_USD_SP500_latest_formatted, "\n")

#### Latest Total USD Value For S&P 500 2X.

In [ ]:
#return the date and Total_USD_SP5002X value from the last row
Total_USD_SP5002X_latest <- SP500_ALL[nrow(SP500_ALL), c("Date", "Total_USD_SP5002X")]
# Format the Total_USD_SP5002X_latest value with commas and periods
Total_USD_SP5002X_latest_formatted <- comma_format(Total_USD_SP5002X_latest$Total_USD_SP5002X)
# Format the date in the Total_USD_SP5002X_latest data frame as a string in the format "YYYY-MM-DD"
Total_USD_SP5002X_latest_date <- format(Total_USD_SP5002X_latest$Date, "%Y-%m-%d")
# Print the formatted date and Total_USD_SP5002X_latest value to the console with the desired format
cat("Date Total_USD_SP5002X\n", Total_USD_SP5002X_latest_date, " ", Total_USD_SP5002X_latest_formatted, "\n")

#### Latest Total USD Value for S&P 500 3X.

In [ ]:
#return the date and Total_USD_SP5003X value from the last row
Total_USD_SP5003X_latest <- SP500_ALL[nrow(SP500_ALL), c("Date", "Total_USD_SP5003X")]
# Format the Total_USD_SP5003X_latest value with commas and periods
Total_USD_SP5003X_latest_formatted <- comma_format(Total_USD_SP5003X_latest$Total_USD_SP5003X)
# Format the date in the Total_USD_SP5003X_latest data frame as a string in the format "YYYY-MM-DD"
Total_USD_SP5003X_latest_date <- format(Total_USD_SP5003X_latest$Date, "%Y-%m-%d")
# Print the formatted date and Total_USD_SP5003X_latest value to the console with the desired format
cat("Date Total_USD_SP5003X\n", Total_USD_SP5003X_latest_date, " ", Total_USD_SP5003X_latest_formatted, "\n")

### Total Number of Months and Years With Lowest USD Value For Each ETF
Latest leverage ETF's values are higher than the 1X S&P 500's value. But<br>
is it always the case that the leverage ETF's values are higher than the<br>
1X S&P 500's value? Let's find out.

Sum counts the number of months on which SP500 is lowest in USD value for<br>
that month. Returns number of months that SP500 is lowest in USD value<br>
for that month.

In [ ]:
Count_Months_SP500_ALL <- data.frame(
  NumMonths_LowestUSD_SP500 = sum(SP500_ALL$Total_USD_SP500 < SP500_ALL$Total_USD_SP5002X & SP500_ALL$Total_USD_SP500 < SP500_ALL$Total_USD_SP5003X),
  NumMonths_LowestUSD_SP5002X = sum(SP500_ALL$Total_USD_SP5002X < SP500_ALL$Total_USD_SP500 & SP500_ALL$Total_USD_SP5002X < SP500_ALL$Total_USD_SP5003X),
  NumMonths_LowestUSD_SP5003X = sum(SP500_ALL$Total_USD_SP5003X < SP500_ALL$Total_USD_SP500 & SP500_ALL$Total_USD_SP5003X < SP500_ALL$Total_USD_SP5002X)
)

Divide each column in Count_Months_SP500_ALL by 12 to get the number of years.
Returns number of years that SP500 is lowest in USD value for that month.

In [ ]:
Count_Years_SP500_ALL <- Count_Months_SP500_ALL / 12

Total number of months in which each ETF had the lowest USD value.

In [ ]:
head(Count_Months_SP500_ALL)

Total number of years in which each ETF had the lowest USD value.

In [ ]:
head(Count_Years_SP500_ALL)

### Percent of Months and Years With Lowest USD Value For Each ETF

In [ ]:
Total_Months_SP500_ALL <- Count_Months_SP500_ALL %>%
  summarise(Count_Months_SP500_ALL$NumMonths_LowestUSD_SP500 + Count_Months_SP500_ALL$NumMonths_LowestUSD_SP5002X + Count_Months_SP500_ALL$NumMonths_LowestUSD_SP5003X)
Percent_months_SP500_lowestUSD <- round((Count_Months_SP500_ALL$NumMonths_LowestUSD_SP500 / Total_Months_SP500_ALL)*100, digits = 2)
Percent_months_SP5002X_lowestUSD <- round((Count_Months_SP500_ALL$NumMonths_LowestUSD_SP5002X / Total_Months_SP500_ALL)*100, digits = 2)
Percent_months_SP5003X_lowestUSD <- round((Count_Months_SP500_ALL$NumMonths_LowestUSD_SP5003X / Total_Months_SP500_ALL)*100, digits = 2)

Percentage time in which 1X S&P 500 ETF had the lowest USD value.

In [ ]:
print(Percent_months_SP500_lowestUSD)

Percentage time in which 2X S&P 500 ETF had the lowest USD value.

In [ ]:
print(Percent_months_SP5002X_lowestUSD)

Percentage time in which 3X S&P 500 ETF had the lowest USD value.

In [ ]:
print(Percent_months_SP5003X_lowestUSD)

1X S&P 500 ETF had the highest percentage time in which it had the lowest<br>
USD value at 96.73% of the time. 2X S&P 500 ETF had the lowest percentage<br>
time in which it had the lowest USD value at 0.00% of the time. 3X S&P<br>
500 ETF had the second highest percentage time in which it had the lowest<br>
USD value at 3.27% of the time.


### Calculate Beta for 2X S&P 500 and 3X S&P 500.
Beta is a measure of a stock's volatility in relation to the overall<br>
market. It is used to determine the risk of a stock or portfolio in<br>
relation to the market as a whole. A beta of 1 indicates that the stock's<br>
price will be just as volatile as the market, while a beta greater than 1<br>
indicates that the stock is more volatile than the market, and a beta<br>
less than 1 indicates that the stock is less volatile than the market.<br>
Note that ETF's are not stocks, but they are traded like stocks. Beta<br>
can be applied just as well to ETF's as it can to stocks.

Select the Date and SP500_DailyPercent for Beta calculation.

In [ ]:
SP500_USD_Beta_Select <- Lev_Percent_df %>%
  select(Date, SP500_DailyPercent)

SSO_USD_Beta_Select <- Lev_Percent_df %>% 
  select(Date, SP500_2X_DailyPercent)

UPRO_USD_Beta_Select <- Lev_Percent_df %>% 
  select(Date, SP500_3X_DailyPercent)

Create a new variable with the converted date values

In [ ]:
SP500_date_covert <- as.POSIXct(SP500_USD_Beta_Select$Date)
SSO_date_covert <- as.POSIXct(SSO_USD_Beta_Select$Date)
UPRO_date_covert <- as.POSIXct(UPRO_USD_Beta_Select$Date)

Convert SP500_data_clean to xts using the new date variable.

In [ ]:
SP500_xts <- xts(SP500_USD_Beta_Select$SP500_DailyPercent, order.by = SP500_date_covert)
SSO_xts <- xts(SSO_USD_Beta_Select$SP500_2X_DailyPercent, order.by = SSO_date_covert)
UPRO_xts <- xts(UPRO_USD_Beta_Select$SP500_3X_DailyPercent, order.by = UPRO_date_covert)

Calculate beta using the CAPM.beta().

In [ ]:
Beta_SSO <- round(CAPM.beta(SSO_xts, SP500_xts),2)
Beta_UPRO <- round(CAPM.beta(UPRO_xts, SP500_xts),2)

Beta_SSO and Beta_UPRO

In [ ]:
Beta_SSO_UPRO <- data.frame(Beta_SSO, Beta_UPRO)

In [ ]:
print(Beta_SSO_UPRO)

Suprisingly, but at the same time not suprisingly. Beta for SSO is 1.97<br>
and Beta for UPRO is 2.97. Which is the same as the median leverage for<br>
SSO and UPRO. This means that leverage is the same as the beta. Thus, 2X<br>
S&P 500 ETF is about twice as volatile as the 1X S&P 500 and 3X S&P 500<br>
ETF is about three times as volatile as the 1X S&P 500.

## Act
Act is the last step in the data analysis process. Act is when you give<br>
high-level recommendations based on the analysis.

### Conclusion
It is often discussed that one should not invest in leveraged ETFs for<br>
long periods of time. The argument is as follows: "If you invest in a<br>
leveraged ETF for a long period of time, you will lose money because of<br>
the daily rebalancing." The first argument is true, but only in the short<br>
term. The market has a slight positive bias over the long term. Therefore,<br>
over the long term, positive upward bias will compensate for the loss by<br>
daily rebalancing.<br>

Another common argument is that, "if you invest a lump sum amount of<br>
money in a leveraged ETF and the market goes down Y percent, it will take<br>
longer to recover than if you invested in a non-leveraged ETF." The<br>
second argument is also true, but most active investors do not invest a<br>
lump sum amount of money into an ETF, then never invest again. It is more<br>
financially prudent to invest by dollar cost averaging. Dollar cost<br>
averaging is when you invest a fixed amount of money at regular intervals<br>
over a long period of time. Dollar cost averaging mediates risk by<br>
spreading out your investment over time. Rather than investing a lump sum<br>
amount of money at one time. Dollar cost averaging has a compounding<br>
effect on your investment. Add the positive bias with the compounding<br>
effect of dollar cost averaging and leverage S&P 500 ETF's outperform<br>
non-leverage ETF's over the long term.<br>  

The investment start date is March 5, 1957. Which aligns with the<br>
introduction of the S&P 500 index. From 1957 to 2023, the non-leverage<br>
S&P 500 ETF had the lowest USD value 90 plus percent of the time. The 2X<br>
leverage S&P 500 ETF had the lowest USD value 0 percent of the time. The<br>
3X leverage S&P 500 ETF had the lowest USD value less than 10 percent of<br>
the time. Note that this is monthly data. Which means that at the<br>
beginning of each month, the 1X S&P 500 ETF had the lowest USD value 90<br>
plus percent of the time. At the beginning of each month, the 2X leverage<br>
S&P 500 ETF had the lowest USD value 0 percent of the time. At the<br>
beginning of each month, the 3X leverage S&P 500 ETF had the lowest USD<br>
value less than 10 percent of the time. Therefore, if an investor is<br>
indifferent by how much they want to outperform the market, but wants to<br>
outperform the market most of the investment time, then the 2X S&P 500<br>
leverage ETF is the way to go. If an investor wants to have the highest<br>
return with the rare risk of a below than average market return at a<br>
small part of the investment time, then the 3X S&P 500 leverage ETF is<br>
the way to go.<br>

Risk factor to consider. Prices for leverage S&P 500 ETF's are not based<br>
on actual historical data for leverage S&P 500 ETF's, as leverage S&P 500<br>
ETF's did not exist when the S&P 500 index was created. Therefore, in<br>
this analysis, leverage S&P 500 ETF's are based on a simple mathematical<br>
model. Since leverage S&P 500 ETF's rebalance daily. Each day has a<br>
slightly different leverage. This slight difference in leverage can cause<br>
the leverage S&P 500 ETF's to deviate from the target leverage of 2X and<br>
3X. Thus, to simulate leverage S&P 500 ETF's, the median leverage from<br>
SSO and UPRO was used. The median leverage for SSO is 1.97 and the median<br>
leverage for UPRO is 2.97. Median leverage is then multiplied by the S&P<br>
500 daily return to simulate the daily return for leverage S&P 500 ETF's.<br>
This simple mathematical model is an approximation of the actual daily<br>
return for leverage S&P 500 ETF's. The approximation is not perfect, but<br>
it is close enough to provide a good estimate of the daily return for<br>
leverage S&P 500 ETF's. If the client is not comfortable with this<br>
approximation, then at the request of the client. More complex<br>
mathematical models can be used to simulate the daily return for leverage<br>
S&P 500 ETF's.<br>

What if the 1X S&P 500 drops more than 33.33%? For a 3X leverage S&P 500.<br>
A drop of more than 33.33% would wipe out the leverage 3X leverage S&P 500.<br>
That would be true if it happens, but if the 1X S&P 500 index drops more<br>
than 20%. Circuit breakers halt trading for the remainder of the day.<br>
Assuming a hypothetical scenario where 3X leverage S&P 500 starts at some<br>
Y price level. Set that Y price level to 100%. The 1X S&P 500 would have<br>
to drop 20% for 9 straight trading days before price level for 3X<br>
leverage S&P 500 drops to about 1%. The 1X S&P 500 dropping 20% for 9<br>
straight trading days has never happened. Though not completely<br>
impossible, but it would have to be some truly apocalyptic event for the<br>
1X S&P 500 to drop 20% for 9 straight trading days. Therefore,<br>
notwithstanding an asteroid strike, nuclear war, or some other<br>
cataclysmic event that could cause 1X S&P 500 to drop 20% for 9 straight<br>
trading days. I assume that 3X leverage S&P 500 to be safe from complete<br>
liquidation by extreme negative drops. If the 3X leverage S&P 500 can<br>
survive extreme drops, then it follows that the 2X can also survive<br>
extreme negative drops in the stock market.<br> 

To conclude, 3X leverage ETF provide the highest return, followed by 2X<br>
leverage ETF and then 1X ETF. However, 3X leverage ETF also provide the<br>
highest volatility, followed by 2X leverage ETF, and then 1X ETF. 3X<br>
leverage ETF are about three times as volatile as the S&P 500 and 2X<br>
leverage ETF are about twice as volatile as the S&P 500. 1X ETF are about<br>
as volatile as the S&P 500 index. 3X leverage ETF are the best choice for<br>
investors who are willing to take on the highest volatility for the<br>
highest return. 2X leverage ETF are the best choice for investors who are<br>
willing to take on a moderate volatility for a moderate return. 1X ETF<br>
are the best choice for investors who are willing to take on the lowest<br>
volatility for the lowest return.<br>

FIN.<br>

References<br>

Cooper, Tony, Alpha Generation and Risk Smoothing Using Managed Volatility (August 25, 2010).<br>
    Available at SSRN: https://ssrn.com/abstract=1664823 or http://dx.doi.org/10.2139/ssrn.1664823

The Vanguard Group. (n.d.). Market volatility regulations. Vanguard.<br>
    https://personal.vanguard.com/us/content/Funds/FundsToolsCircuitBreakersJSP.jsp<br>

Valetkevitch, C. (2013, April 10). TIMELINE-Key dates and milestones in the S&P 500’s history. U.S.<br>
    https://www.reuters.com/article/usa-stocks-sp-timeline/timeline-key-dates-and-milestones-in-the-sp-500s-history-idUSL2N0CX13620130410<br>